**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [28]:
!pip install sk-video

In [29]:
!pip install opencv-python

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is used to mix exploration and exploitation. With no ```epsilon``` there would be no exploration! With a probability ```epsilon``` we explore (try randomly any of the different possible actions, so that the algorithm will discover which one is the best) and with the probabiliy 1 - ```epsilon``` exploit using the best action according to our algorithm so far.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=21 # set small when debugging
epochs_test=21 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

*board* contains for each cell the reward (cheese or poison).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        #pass
        return np.random.randint(4, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
          # The agent performs an action
          action = agent.learned_act(state) 

          # Apply an action to the environment, get the next state, the reward
          # and if the games end
          prev_state = state
          state, reward, game_over = env.act(action)

          # Update the counters
          if reward > 0:
              win = win + reward
          if reward < 0:
              lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [37]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/14.0. Average score (-2.0)
Win/lose count 11.5/6.0. Average score (1.75)
Win/lose count 9.0/12.0. Average score (0.16666666666666666)
Win/lose count 9.5/12.0. Average score (-0.5)
Win/lose count 11.0/18.0. Average score (-1.8)
Win/lose count 9.0/9.0. Average score (-1.5)
Win/lose count 13.5/10.0. Average score (-0.7857142857142857)
Win/lose count 11.5/12.0. Average score (-0.75)
Win/lose count 12.0/10.0. Average score (-0.4444444444444444)
Win/lose count 7.5/11.0. Average score (-0.75)
Win/lose count 13.5/16.0. Average score (-0.9090909090909091)
Win/lose count 7.0/8.0. Average score (-0.9166666666666666)
Win/lose count 13.5/11.0. Average score (-0.6538461538461539)
Win/lose count 7.0/10.0. Average score (-0.8214285714285714)
Win/lose count 13.5/7.0. Average score (-0.3333333333333333)
Win/lose count 7.0/8.0. Average score (-0.375)
Win/lose count 10.0/15.0. Average score (-0.6470588235294118)
Win/lose count 11.5/14.0. Average score (-0.75)
Win/lose count 10.0/24.0. 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




First equality is just the Bellmann equation:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> = r(s,a)+ \gamma \sum_{s'} \pi(a|s)p(s',a'|s,a)E_{p^{\pi}}[\sum_{t=1}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'] \> = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] .
\end{equation*}

Second equality is Bellman equation for optimal policy:

\begin{equation*}Q^*(s,a)=E_{p^{\pi^*}}[\sum_{t=0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> = r(s,a)+ \gamma \sum_{s'} \pi^*(a|s)p(s'|s,a)E_{p^{\pi^*}}[\sum_{t=1}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'] \> = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{*}(s',a')] .
\end{equation*}

To obtain the third equality, we just inject our last equality inside 
\begin{equation*}\mathcal{L}(\theta)=E_{s,a}||Q^*(s,a,\theta)-Q(s,a,\theta)||^2 \end{equation*} and then take Q as a proxy for $Q^*$ since we do not have $Q^*$. (Note that there should be only one "max")

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)+5>self.max_memory:
          for i in range(5): #remove first four elements
            self.memory.pop(0)
        self.memory=self.memory+m #concatenate our batch in our memory
        pass

    def random_access(self):
        num=5*np.random.randint((len(self.memory)/5), size=1)[0]
        return [self.memory[num],self.memory[num+1],self.memory[num+2],self.memory[num+3],self.memory[num+4]]
        #pass

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #pass 
        a= self.model.predict(s.reshape([1,s.shape[0],s.shape[1],self.n_state])) #s.shape[2]
        return np.argmax(a) 

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            pool=self.memory.random_access() 
            game_over=pool[4]

            if game_over_:
                ######## FILL IN
                target_q[i,pool[2]]=pool[3]
            else:
                ######## FILL IN
                s=pool[1]
                target_q[i,pool[2]]=pool[3]+self.discount*np.amax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],self.n_state]))) #s.shape[2]
               
            input_states[i,:,:,:] = pool[0]
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q) 


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse") 
        self.model = model

            
class DQN_FC(DQN):
  def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)

        model = Sequential()
        model.add(Dense(16, input_shape=(5,5,self.n_state)))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(output_dim=4))
        model.add(Activation('linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")  
        self.model = model

        
  
        

In [41]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.05, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=4)`







Epoch 000/021 | Loss 0.0097 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/021 | Loss 0.0141 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/021 | Loss 0.0078 | Win/lose count 5.5/8.0 (-2.5)
Epoch 003/021 | Loss 0.0123 | Win/lose count 2.5/5.0 (-2.5)
Epoch 004/021 | Loss 0.0142 | Win/lose count 5.5/11.0 (-5.5)
Epoch 005/021 | Loss 0.0082 | Win/lose count 6.5/6.0 (0.5)
Epoch 006/021 | Loss 0.0133 | Win/lose count 4.0/6.0 (-2.0)
Epoch 007/021 | Loss 0.0242 | Win/lose count 4.0/7.0 (-3.0)
Epoch 008/021 | Loss 0.0054 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/021 | Loss 0.0245 | Win/lose count 9.5/8.0 (1.5)
Epoch 010/021 | Loss 0.0225 | Win/lose count 2.0/4.0 (-2.0)
Epoch 011/021 | Loss 0.0096 | Win/lose count 4.5/3.0 (1.5)
Epoch 012/021 | Loss 0.0045 | Win/lose count 7.0/6.0 (1.0)
Epoch 013/021 | Loss 0.0102 | Win/lose count 4.5/3.0 (1.5)
Epoch 014/021 | Loss 0.0079 | Win/lose count 5.5/3.0 (2.5)
Epoch 015/021 | Loss 0.0033 | Win/lose count 4.0/3.0 (1.0)
Epoch 016/021 | Loss 0.0018 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN

        model = Sequential()
        model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(5,5,self.n_state)))
        model.add(MaxPooling2D())
        model.add(Dropout(0.5))
        model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
        model.add(AveragePooling2D(1)) #model.add(MaxPooling2D(2)) 
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(output_dim=4, activation='sigmoid'))

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [43]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=4)`


Epoch 000/021 | Loss 0.0350 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/021 | Loss 0.0065 | Win/lose count 4.5/2.0 (2.5)
Epoch 002/021 | Loss 0.0071 | Win/lose count 3.5/2.0 (1.5)
Epoch 003/021 | Loss 0.0063 | Win/lose count 3.0/0 (3.0)
Epoch 004/021 | Loss 0.0008 | Win/lose count 2.5/3.0 (-0.5)
Epoch 005/021 | Loss 0.0024 | Win/lose count 2.0/4.0 (-2.0)
Epoch 006/021 | Loss 0.0012 | Win/lose count 1.5/2.0 (-0.5)
Epoch 007/021 | Loss 0.0042 | Win/lose count 3.0/3.0 (0.0)
Epoch 008/021 | Loss 0.0011 | Win/lose count 1.5/0 (1.5)
Epoch 009/021 | Loss 0.0048 | Win/lose count 5.0/5.0 (0.0)
Epoch 010/021 | Loss 0.0098 | Win/lose count 1.5/3.0 (-1.5)
Epoch 011/021 | Loss 0.0090 | Win/lose count 2.5/7.0 (-4.5)
Epoch 012/021 | Loss 0.0050 | Win/lose count 2.0/2.0 (0.0)
Epoch 013/021 | Loss 0.0059 | Win/lose count 1.5/2.0 (-0.5)
Epoch 014/021 | Loss 0.0087 | Win/lose count 0.5/4.0 (-3.5)
Epoch 015/021 | Loss 0.0021 | Win/lose count 2.5/1.0 (1.5)
Epoch 016/021 | Loss 0.0107 | Win/lose count 2.0/3.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [44]:
env = Environment(grid_size=size, max_time=T,temperature=0.4)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=4)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=4)`


Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 5.0/0. Average score (3.25)
Win/lose count 2.5/1.0. Average score (2.6666666666666665)
Win/lose count 3.0/0. Average score (2.75)
Win/lose count 1.5/0. Average score (2.5)
Win/lose count 0/0. Average score (2.0833333333333335)
Win/lose count 3.0/0. Average score (2.2142857142857144)
Win/lose count 1.0/0. Average score (2.0625)
Win/lose count 1.5/1.0. Average score (1.8888888888888888)
Win/lose count 4.0/0. Average score (2.1)
Win/lose count 0.5/2.0. Average score (1.7727272727272727)
Win/lose count 2.5/0. Average score (1.8333333333333333)
Win/lose count 3.0/1.0. Average score (1.8461538461538463)
Win/lose count 3.5/1.0. Average score (1.8928571428571428)
Win/lose count 1.5/1.0. Average score (1.8)
Win/lose count 2.5/0. Average score (1.84375)
Win/lose count 3.5/1.0. Average score (1.8823529411764706)
Win/lose count 0/0. Average score (1.7777777777777777)
Win/lose count 2.0/0. Average score (1.7894736842105263)
Wi

In [45]:
HTML(display_videos('cnn_test10.mp4'))

In [46]:
HTML(display_videos('fc_test10.mp4'))

CNN is better than FC. Issue I observe: at some point, the rat loops...we should try and mike that rat explore the map. Moreover score is better when temperature is higher: this is normal since there are more reward. When temperature is higher, the moment when the rat starts to loop undefinitely is later then with low temperatures.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
class Agent(object):
    def __init__(self, epsilon, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

def test_explore(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
          # The agent performs an action
          action = agent.learned_act(state) 

          # Apply an action to the environment, get the next state, the reward
          # and if the games end
          prev_state = state
          state, reward, game_over = env.act(action, train=True)

          # Update the counters
          if reward > 0:
              win = win + reward
          if reward < 0:
              lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # decreasing eps greedy explo:
        if agent.epsilon>=0.1:
          agent.set_epsilon(agent.epsilon-0.02)
        print(agent.epsilon)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train):
        """This function returns the new state, reward and decides if the
        game ends.""" 

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
          
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]  
        return state
    

In [0]:
epochs_train=100
epochs_test=100

In [49]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 64,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=4)`


0.98
Epoch 000/100 | Loss 0.0249 | Win/lose count 7.5/28.10000000000011 (-20.60000000000011)
0.96
Epoch 001/100 | Loss 0.0204 | Win/lose count 8.8/25.500000000000085 (-16.700000000000085)
0.94
Epoch 002/100 | Loss 0.0151 | Win/lose count 6.200000000000001/26.80000000000012 (-20.600000000000122)
0.9199999999999999
Epoch 003/100 | Loss 0.0246 | Win/lose count 5.6000000000000005/26.500000000000092 (-20.90000000000009)
0.8999999999999999
Epoch 004/100 | Loss 0.0275 | Win/lose count 9.600000000000003/29.700000000000106 (-20.1000000000001)
0.8799999999999999
Epoch 005/100 | Loss 0.0295 | Win/lose count 5.2/29.800000000000164 (-24.600000000000165)
0.8599999999999999
Epoch 006/100 | Loss 0.0139 | Win/lose count 3.1999999999999997/29.20000000000012 (-26.00000000000012)
0.8399999999999999
Epoch 007/100 | Loss 0.0138 | Win/lose count 6.100000000000001/33.50000000000014 (-27.40000000000014)
0.8199999999999998
Epoch 008/100 | Loss 0.0243 | Win/lose count 8.000000000000002/25.1000000000001 (-17.1000

In [52]:
# Evaluation
test_explore(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 1.6/19.70000000000001. Average score (-18.10000000000001)
Win/lose count 0.8/21.90000000000004. Average score (-19.600000000000023)
Win/lose count 0.4/20.000000000000014. Average score (-19.60000000000002)
Win/lose count 0.8/19.900000000000013. Average score (-19.47500000000002)
Win/lose count 0.8/21.90000000000004. Average score (-19.800000000000022)
Win/lose count 0/21.100000000000033. Average score (-20.01666666666669)
Win/lose count 0.8/19.900000000000013. Average score (-19.885714285714307)
Win/lose count 2.4/21.500000000000036. Average score (-19.787500000000023)
Win/lose count 0/25.100000000000094. Average score (-20.37777777777781)
Win/lose count 0.4/20.000000000000014. Average score (-20.30000000000003)
Win/lose count 0.8/22.900000000000055. Average score (-20.463636363636393)
Win/lose count 0.4/23.000000000000057. Average score (-20.641666666666698)
Win/lose count 1.2000000000000002/23.80000000000007. Average score (-20.792307692307727)
Win/lose count 2.0/20.60

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***